In [2]:
from transformers import pipeline


In [3]:
qa_pipeline = pipeline(task="question-answering", 
                       model='twmkn9/albert-base-v2-squad2', 
                       tokenizer='twmkn9/albert-base-v2-squad2',
                       framework='pt')

In [4]:
output = qa_pipeline(context="I am a company", question="When is the bill due?", handle_impossible_answer=True)


In [5]:
print(output)

{'score': 0.999964714050293, 'start': 0, 'end': 0, 'answer': ''}


In [ ]:
def get_answers(sorted_i_j, tokens, k):
    counter = 0
    answers_indices = [] # i, j pairs of answers
    answers = [] # string answers 
    while len(answers_indices) < k and counter < len(sorted_i_j):
        pair = sorted_i_j[counter]
        if pair[1] >= pair[0]: #end token after start token
            if counter == 0: # first i,j
                answer = tokens[pair[0]] #first token (i)
                for i in range(pair[0] + 1, pair[1] + 1):
                    if tokens[i][0:1] != '▁':
                        answer += tokens[i]
                    else:
                        answer += ' ' + tokens[i]
                answers_indices.append(sorted_i_j[counter])
                answers.append(answer)
            elif (pair[0] >= sorted_i_j[counter-1][1]) or (sorted_i_j[counter-1][0] >= pair[1]):
                # ^start token of current span is after end token of previous span; end token of current span is before start of prev
                answer = tokens[pair[0]]
                for i in range(pair[0] + 1, pair[1] + 1):
                    if tokens[i][0:1] != '▁':
                        answer += tokens[i]
                    else:
                        answer += ' ' + tokens[i]
                answers_indices.append(sorted_i_j[counter])
                answers.append(answer)
        counter += 1
    if len(answers_indices) < k:
        while len(answers_indices) < k:
            answers_indices.append((-1, -1))
    for i in answers_indices:
        if i == (-1, -1):
            answers.append('')
    return answers

In [ ]:
def get_score_dict(start_scores, end_scores):
    '''
    i = start score index
    j = end score index
    score = sum of start_scores[i] and end_scores[j]
    '''
    start_scores_copy = start_scores.copy()
    end_scores_copy = end_scores.copy()
    i_j_scores = {}
    for x in range(len(start_scores)):
        score = 0
        i = np.argmax(start_scores_copy)
        score += start_scores_copy[i]
        start_scores_copy[i] = 0
        j = np.argmax(end_scores_copy)
        score += end_scores_copy[j]
        end_scores_copy[j] = 0
        i_j_scores[(i, j)] = score
    return i_j_scores